# 하기 전에!!! 
혹시 attribute error가 뜬다면 해당 module py파일에 들어가서

import tensorflow as tf를

import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

이 두줄로 다 고치도록 하자! 정말 만능 코드이다!

In [2]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install keras==2.2.0

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tensorflow==2.2.0 --user

  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4835 sha256=a1dbd18885cb5ca940a6f556c677a3f426ae14f991c584daefd8513aad886eda
  Stored in directory: c:\users\kevin\appdata\local\pip\cache\wheels\3f\e3\ec\8a8336ff196023622fbcb36de0c5a5c218cbb24111d1d4c7f2
Successfully built termcolor
Note: you may need to restart the kernel to use updated packages.


ERROR: keras 2.2.0 has requirement keras-preprocessing==1.0.1, but you'll have keras-preprocessing 1.1.2 which is incompatible.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install pyserial

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install easygui

Note: you may need to restart the kernel to use updated packages.


/home/"사용자이름"/anaconda3/envs/"프로젝트이름"/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py  
여기 tensorflow_backend.py를 고쳐주면 된다.
get_default_graph를 아마 tf.get_default_graph 로 사용하고 있을 것이다.
이것을 tf.compat.v1.get_default_graph 로 바꿔야한다.
그리고 저장.
만약 여기서 다른 텐서플로우를 사용할 경우에는 다시 바꿔야한다.

이 밑에가 진짜 코드!

In [3]:
import time as ti
import winsound as win
import numpy as np
from math import pi, cos, sin
import serial
import easygui
import tkinter as tk
import cv2
import datetime
import os

# 실행하기 전 port가 맞는지 확인!
PORT=easygui.enterbox('포트가 COM 몇 번에 연결되어있나요? ex) 9')
PORT2='COM'+str(PORT)
BaudRate = 9600
ARD=serial.Serial(PORT2,BaudRate)

easygui.msgbox('해저 자원 머신러닝 프로그램')
r=float(easygui.enterbox('인식 거리(cm)를 입력하세요. :'))

def Ardread():
    try:
        LINE=ARD.readline()
        code0=LINE.decode('utf-8')[:-2].split(',')
        return code0
    except:
        pass

def Ardcount():
    while True:
        s_time=ti.time()
        ans=Ardread()
        f_time=ti.time()
        timer=f_time-s_time
        if timer>0.1:
            return ans
            break
        else:
            continue

while True:
    s=float(Ardcount()[0])
    print(s)
    if 10<s<=r:   
        for u in range(1):
            win.Beep(5000,750)
            win.Beep(3000,750)
        easygui.msgbox('장애물이 발견되었습니다. 암석 촬영을 시작합니다.')

        #0번은 캠, 따로 캠을 연결하면 1번으로 바꾸기
        video_capture = cv2.VideoCapture(1)

        grabbed, frame = video_capture.read()
        cv2.imshow('Original Video', frame)
        cv2.waitKey(10)

        file = datetime.datetime.now().strftime("%Y%m%d_%H%M%S%f") + '.png'
        cv2.imwrite(os.path.join("c:/testimage/subimage3/", str(file)), frame)
        easygui.msgbox(file, ' saved',image='c:/testimage/subimage3/'+str(file))
        #easygui.msgbox('file name : '+file+' saved')

        video_capture.release()
        cv2.destroyAllWindows()
        askmore=easygui.buttonbox('촬영이 완료되었습니다. 계속 탐사를 진행하시겠습니까?',choices=['y','n'])
        if askmore=='n': break
        else: continue
            
            
from PIL import Image
import os, glob, numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

caltech_dir = "C:/testimage"
categories = ["hwa","hyun","jum","kyu","suk"]
nb_classes = len(categories)

image_w = 64
image_h = 64

pixels = image_h * image_w * 3

X = []
y = []

for idx, cat in enumerate(categories):
    
    #one-hot 돌리기.
    label = [0 for i in range(nb_classes)]
    label[idx] = 1

    image_dir = caltech_dir + "/" + cat
    print(image_dir)
    files = glob.glob(image_dir+"/*.png")
    print(cat, " 파일 길이 : ", len(files))
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)

        X.append(data)
        y.append(label)

        if i % 700 == 0:
            print(cat, " : ", f)

X = np.array(X)
y = np.array(y)
#1 0 0 0 이면 airplanes
#0 1 0 0 이면 buddha 이런식


X_train, X_test, y_train, y_test = train_test_split(X, y)
xy = (X_train, X_test, y_train, y_test)
np.save("./multi_image_data.npy", xy)

print("ok", len(y))

import os, glob, numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import keras.backend.tensorflow_backend as K

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

X_train, X_test, y_train, y_test = np.load('./multi_image_data.npy', allow_pickle=True)
#print(X_train.shape)
#print(X_train.shape[0])

categories = ["hwa","hyun","jum","kyu","suk"]
nb_classes = len(categories)

#일반화
X_train = X_train.astype(float) / 255
X_test = X_test.astype(float) / 255

with K.tf_ops.device('/device:GPU:0'):
    model = Sequential()
    model.add(Conv2D(32, (3,3), padding="same", input_shape=X_train.shape[1:], activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3,3), padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model_dir = './model'
    
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
    
    model_path = model_dir + '/multi_img_classification.model'
    checkpoint = ModelCheckpoint(filepath=model_path , monitor='val_loss', verbose=1, save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', patience=6)
    
model.summary()

#데이터셋이 적어서 validation을 그냥 test 데이터로 했습니다. 
#데이터셋이 충분하시면 이렇게 하시지 마시고 validation_split=0.2 이렇게 하셔서 테스트 셋으로 나누시길 권장합니다.
history = model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test), callbacks=[checkpoint, early_stopping])

print("정확도 : %.4f" % (model.evaluate(X_test, y_test)[1]))

y_vloss = history.history['val_loss']
y_loss = history.history['loss']

x_len = np.arange(len(y_loss))

plt.plot(x_len, y_vloss, marker='.', c='red', label='val_set_loss')
plt.plot(x_len, y_loss, marker='.', c='blue', label='train_set_oss')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('loss')
plt.grid()
plt.show()

from PIL import Image
import os, glob, numpy as np
from keras.models import load_model

caltech_dir = "c:/testimage/subimage3"
image_w = 64
image_h = 64

pixels = image_h * image_w * 3

X = []
filenames = []
files = glob.glob(caltech_dir+"/*.png")
for i, f in enumerate(files):
    img = Image.open(f)
    img = img.convert("RGB")
    img = img.resize((image_w, image_h))
    data = np.asarray(img)
    filenames.append(f)
    X.append(data)

X = np.array(X)
model = load_model('./model/multi_img_classification.model')

prediction = model.predict(X)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
cnt = 0

#이 비교는 그냥 파일들이 있으면 해당 파일과 비교. 카테고리와 함께 비교해서 진행하는 것은 _4 파일.
for i in prediction:
    pre_ans = i.argmax()  # 예측 레이블
    #print(i)
    #print(pre_ans)
    pre_ans_str = ''
    if pre_ans == 0: pre_ans_str = "화강편마암"
    elif pre_ans == 1: pre_ans_str = "현무암"
    elif pre_ans == 2: pre_ans_str = "점판암"
    elif pre_ans == 3: pre_ans_str = "규암"
    elif pre_ans == 4: pre_ans_str = "석회암"
    #else: pre_ans_str = "암초"
    if i[0] >= 0.8 : print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[1] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"으로 추정됩니다.")
    if i[2] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[3] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[4] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    print('')
    #if i[5] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    cnt += 1
    # print(i.argmax()) #얘가 레이블 [1. 0. 0.] 이런식으로 되어 있는 것을 숫자로 바꿔주는 것.
    # 즉 얘랑, 나중에 카테고리 데이터 불러와서 카테고리랑 비교를 해서 같으면 맞는거고, 아니면 틀린거로 취급하면 된다.
    # 이걸 한 것은 _4.py에.
    
        

SerialException: could not open port 'COMNone': FileNotFoundError(2, '지정된 파일을 찾을 수 없습니다.', None, 2)